In [13]:
import os
import sys
import random
import math
from copy import copy, deepcopy
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import scipy
import pandas as pd
import pixiedust
from pathlib import Path
# load numpy array from npy file
from numpy import load
from scipy.stats import multivariate_normal
from scipy.optimize import linear_sum_assignment
import heapq
import scipy.linalg as la

In [58]:
# HERE IS WHERE I CHANGE VIDEO LOCATION AND TRACKER TYPE FOR SAVING AND LOADING DATA:
# Specific video location
Video = "Short_Garden"
# Tracking type
Tracker = "_hung_area"
# Video using mask or tracking output
vid_input_type = "original" #Choose between masked or original
# NOW EVERYTHING ELSE GETS DONE AUTOMATICALLY

# Root directory of the project
ROOT_DIR = os.path.abspath("C:/Users/ddefr/OneDrive/Documents/Skripsie/Mask_RCNN")

# Directory for all videos
all_vids = Path("C:/Users/ddefr/OneDrive/Documents/skripsie_videos")

#Specific video dir
video_dir = all_vids / Video

# Measurements to load:
load_measurement = video_dir / "measurements.npy"

# Result save locations:
mu_save = os.path.join(video_dir,'mu{}.npy'.format(Tracker))
cov_save = os.path.join(video_dir,'cov{}.npy'.format(Tracker))
mu_obs_save = os.path.join(video_dir,'mu_obs{}.npy'.format(Tracker))
cov_obs_save = os.path.join(video_dir,'cov_obs{}.npy'.format(Tracker))
number_of_detections_save = os.path.join(video_dir,'number_of_detections{}.npy'.format(Tracker))

# Using mask pictures or tracking pictures to make video:
image_path_save = os.path.join(video_dir,'{}_tracking{}_pics'.format(vid_input_type,Tracker))
image_path_mask = video_dir / "mask_pics/"

# video input
video_input = os.path.join(video_dir,'{}.mp4'.format(vid_input_type))

# video output
video_output = os.path.join(video_dir,'{}_tracking{}.mp4'.format(vid_input_type,Tracker))
video_output_mask = os.path.join(video_dir,'masked.mp4')

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco
%matplotlib inline

In [59]:
#graph drawing:
#Multivariate functions
def multi_gauss_pdf(mu, covariance, x, dim):
    if np.all(mu != None) and np.all(covariance != None) and np.all(x != None):
        x_m = x - mu
        ans = (1./(np.sqrt(((2*np.pi)**dim)*np.linalg.det(covariance))))*np.exp(-0.5*(np.linalg.solve(covariance,x_m).T.dot(x_m)))
        x_0 = mu - mu
        c_norm = (1./(np.sqrt(((2*np.pi)**dim)*np.linalg.det(covariance))))*np.exp(-0.5*(np.linalg.solve(covariance,x_0).T.dot(x_0)))
        ans_norm = ans/c_norm
    else:
        ans = None
        ans_norm = None
    return (ans_norm)

#Create grid and multivariate normal
def Bivariate_plot(x_range,y_range,mu,covariance,levels):
    x = np.linspace(0,x_range,x_range)
    y = np.linspace(0,y_range,y_range)
    X, Y = np.meshgrid(x,y)
    pos = np.array([X.flatten(),Y.flatten()]).T
    rv = multivariate_normal([mu[0][0],mu[1][0]], covariance)
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111)
    plt.title('Bivariate Gaussian distribution')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    CS = ax.contour(rv.pdf(pos).reshape(y_range,x_range), levels,colors=['green', 'red', 'blue'])
    leg = CS.legend_elements()
    labels = ['$p_1(x)$ = %f'%(levels[2]), '$p_2(x)$ = %f'%(levels[1]),'$p_3(x)$ = %f'%(levels[0])]
    ax.legend([leg[0][2], leg[0][1], leg[0][0]], labels)
    return(plt)

In [60]:
# load array
cov_obs = load(cov_obs_save, allow_pickle=True)
# load array
mu_obs = load(mu_obs_save, allow_pickle=True)
# load array
measurements = load(load_measurement, allow_pickle=True)
# load array
number_of_detections = load(number_of_detections_save, allow_pickle=True)

In [61]:
#Set Paths
vid_path = video_input
proccessed_image_path = image_path_save
#Start Capture + get number of frames
capture = cv2.VideoCapture(str(vid_path))
tot_frames = capture.get(7)
frame_count = 0
frames = []
startAngle = 0
endAngle = 360
thickness = 5
colours = np.random.random_integers(0,255, size=(1500,3))
color = np.array([[40,250,25],[250,20,140],[0,20,80],[250,25,50],[25,0,100],[0,0,255],[255,0,0],[100,0,100],[0,255,255],[255,255,0],[0,255,0],[0,100,0],[80,255,80],[255,255,255],[0,0,0]])
while True:
    ret, frame = capture.read()
    # Bail out when the video file ends
    if not ret:
        break        
    # Save each frame of the video to a list
    frame_count += 1
    if frame_count == 20:
        stop = []
    frames.append(frame)   
    for i, item in enumerate(zip(frames)):
        frame = item[0]
        for m in range(len(measurements[frame_count-1])):
            if not np.any(pd.isnull(measurements[frame_count-1][m])):
                coords = (int(measurements[frame_count-1][m][0]), int(1080-measurements[frame_count-1][m][1]))
                dot_size = 5
                black = [0,0,0]
                cv2.circle(frame, coords, dot_size, black, -1)
        
        for z in range(len(mu_obs[frame_count-1])):
            if not np.any(pd.isnull(mu_obs[frame_count-1][z])) and number_of_detections[z] > 5:
                coords = (int(mu_obs[frame_count-1][z][0]), int(1080-mu_obs[frame_count-1][z][1]))
                avg_cov = int((cov_obs[frame_count-1][z][0][0]+cov_obs[frame_count-1][z][1][1])/2)
                std_dev = int(np.sqrt(avg_cov))
                cov_2_by_2 = np.array([[cov_obs[frame_count-1][z][0][0], cov_obs[frame_count-1][z][0][1]],[cov_obs[frame_count-1][z][1][0], cov_obs[frame_count-1][z][1][1]]])
                eigvals, eigvecs = la.eig(cov_2_by_2)
                eigvals = eigvals.real
                if eigvals[0] > eigvals[1]:
                    a = int(np.sqrt(eigvals[0]))
                    b = int(np.sqrt(eigvals[1]))
                    angle = int(np.arctan(eigvecs[0][1]/eigvecs[0][0]))
                else:
                    a = int(np.sqrt(eigvals[1]))
                    b = int(np.sqrt(eigvals[0]))
                    angle = int(np.arctan(eigvecs[1][1]/eigvecs[1][0]))
                axesLength = (int(a),int(b))
                c = [int(colours[z][0]),int(colours[z][1]),int(colours[z][2])]
                #cv2.circle(frame, coords, int(mu_obs[frame_count-1][z][4], c, -1)
                cv2.ellipse(frame, coords, axesLength, angle, 0, 360, c, 2) 
        cv2.imshow('frame', frame)
        name = '{0}.jpg'.format(frame_count)
        name = os.path.join(str(proccessed_image_path), name)
        cv2.imwrite(name, frame)
        #Clear the frames array to start the next batch
    frames = []

C:\Users\ddefr\Anaconda3\envs\pgmpractice\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(0, 255 + 1) instead
  if sys.path[0] == '':
C:\Users\ddefr\Anaconda3\envs\pgmpractice\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in double_scalars


In [62]:
#Set Paths
proccessed_image_path = image_path_save
outvid = video_output
#Start Capture + get number of frames

def make_video(outvid, images=None, fps=20, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

import glob
import os

# Directory of images to run detection on
ROOT_DIR = os.getcwd()
VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")
images = list(glob.iglob(os.path.join(proccessed_image_path, '*.*')))
# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))
make_video(outvid, images, fps=20)



<VideoWriter 000001FAE1AC8DB0>